# Task 2

In [0]:
from pyspark.sql.functions import concat, col, lit 

In [0]:
from pyspark.sql.functions import col, unix_timestamp, to_date
import pyspark.sql.functions as f
dbfs_fileStore_prefix = "/FileStore/tables"
prefix = "ontimeperformance"
size = "small"
year = 2000

In [0]:
def clean_column_names(df):
  tempList = [] #Edit01
  for col in df.columns:
      new_name = col.strip()
      new_name = "".join(new_name.split())
      new_name = new_name.replace('.','') 
      tempList.append(new_name) 

  return df.toDF(*tempList) 


In [0]:
def task_2(spark_session, flights_path, airlines_path, year):
    flights_df = spark_session.read.csv(flights_path,
                                        inferSchema=True,
                                        header=True)
    flights_df = clean_column_names(flights_df).dropna()

    flights_df = flights_df.withColumn('dates',  to_date(unix_timestamp(col('flight_date'), 'yyyy-MM-dd').cast("timestamp")))
    flights_df = flights_df.withColumn('year',f.year(f.to_timestamp('dates', 'dd/MM/yyyy')))
    
    flights_df = flights_df.withColumn('delay_time', flights_df.actual_departure_time - flights_df.scheduled_depature_time)
    
    
    flights_df = flights_df.select([c for c in flights_df.columns if c in {'carrier_code','delay_time', 'year'}]).filter(flights_df.delay_time > 0)
    airlines_df = spark_session.read.csv(airlines_path,
                                        inferSchema=True,
                                        header=True)
    airlines_df = clean_column_names(airlines_df).dropna().filter(airlines_df.country == "United States")
    
    US_flights  = flights_df.join(airlines_df, flights_df.carrier_code == airlines_df.carrier_code, how='inner')
    

    US_flights = US_flights.filter(US_flights.year == year)
    US_flights = US_flights.drop('year')
    delay_count = US_flights.groupBy(US_flights.name).count()
    avg_delay = US_flights.groupBy(US_flights.name).avg()
    min_delay = US_flights.groupBy(US_flights.name).min()
    max_delay = US_flights.groupBy(US_flights.name).max()
    
    join_table = delay_count.join(avg_delay, on = 'name').join(min_delay, on = 'name').join(max_delay, on = 'name')
    result = join_table.toDF('airline_name', 'num_delays', 'average_delay', 'min_delay', 'max_delay')
    
    result = result.orderBy('airline_name')
    
    result = result.select("*", concat(col("airline_name"),lit(' \t '),col("num_delays"),lit(' \t '), col("average_delay"), col("min_delay"),lit(' \t '), col("max_delay")).alias("Result"))
    
    
    # SAVE THE RESULT COLUMN TO CSV 
    resultDf = result.toPandas()
    resultDf = resultDf['Result'].to_frame()
    display(resultDf)
    
    



In [0]:
task_2(spark, f"{dbfs_fileStore_prefix}/{prefix}_flights_{size}.csv", f"{dbfs_fileStore_prefix}/{prefix}_airlines.csv", year)

Result
Alaska Airlines Inc. 64 43.8751 199
American Airlines Inc. 250 55.4281 1946
Continental Air Lines Inc. 124 33.5967741935483841 376
Delta Air Lines Inc. 385 41.948051948051951 2345
Northwest Airlines Inc. 144 53.2291666666666641 397
Southwest Airlines Co. 357 48.7198879551820742 333
US Airways 233 40.729613733905581 574
United Airlines 372 56.048387096774191 1506
